# Walking friction surface creation

Clean up various rasters and combining them into a walking friction surface. This forms half of the multi-modal friction surface and can also be used as a standalone analysis tool (for walking-only analysis). This will later be merged with an on-road friction surface for a final product.

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'

from datetime import datetime

import common_rasterio_ops as rast_ops # custom functions

import numpy as np
from numpy import pi, log, tan, empty, float32, arctan, rad2deg, gradient
from numpy import arctan2, reshape, where
from scipy.ndimage import gaussian_gradient_magnitude

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

import json

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters that will eventually be passed from the UI
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables that will come from UI so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_dem_folder = data_loaded['local_dem_folder']
local_lc_folder = data_loaded['local_lc_folder']
local_lc_folder_non_seasonal = data_loaded['local_lc_folder_non_seasonal']
local_roads_folder = data_loaded['local_roads_folder']
local_water_folder = data_loaded['local_waterways_folder']
local_boundaries_folder = data_loaded['local_boundaries_folder']
force_rivers = data_loaded['force_rivers']
fric_dir = data_loaded['fric_dir']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']
buffer_m = data_loaded['buffer_m']
level = data_loaded['level']
seasonal_lc = data_loaded['seasonal_lc']
if level != 'custom':
    shapefile_adm_field = data_loaded['shapefile_adm_field']
    adm_name = data_loaded['adm_name']


In [ ]:
if seasonal_lc == 1:
    lc_folder_final = local_lc_folder

else: 
    lc_folder_final = local_lc_folder_non_seasonal

    lc_file_generic = sorted([os.path.join(lc_folder_final,file) \
                for file \
                in os.listdir(lc_folder_final) \
                if file.endswith(".tif")])

    lc_file_generic = lc_file_generic[0]
    
seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".txt")])

for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".txt","")   

Load Shapefile of area of interest to clip the final data

In [ ]:
aoi_pth = local_boundaries_folder + level + '\\'

aoi_file = sorted([os.path.join(aoi_pth,file) \
            for file \
            in os.listdir(aoi_pth) \
            if file.endswith(".shp")])

aoi_file = aoi_file[0]

aoi = gpd.read_file(aoi_file)

aoi = aoi[aoi[shapefile_adm_field] == adm_name]
aoi = aoi.to_crs(dest_crs)

# Buffer the polygon so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(buffer_m)

# Processing

Load in and process various input rasters for off-road travel. Note that the decision to use the landcover as the reference layer (in terms of projection, cell size, etc.) is arbitrary and the DEM could easily be used for such instead.

## Reclassify landcover

In [ ]:
# Build a "lookup array" where the index is the original value and the value
# is the reclassified value.  Setting all of the reclassified values is cheap 
# because the memory is only allocated once for the lookup array.
# Replacement values are the divisors of walking speeds specific to the landcover classes specified in the file "class_speed_modifiers.csv" in your inputs landcover folder
# THESE ARE EXAMPLE VALUES SET FOR GOOGLE DYNAMIC WORLD DATASETS AND MUST BE REPLACED FOR YOUR CONTEXT 
# Always verify that the landcover class values in your rasters correspond correctly to the raster_value in the .csv file
# There are different land cover class numbering schemes between different datasets, so always check
# refer to "Spatial Analysis by Cost Functions" by Irmela Herzog (2020) for guidance on modifier values

lookup = np.arange(256, dtype=np.float32)
lookup[:] = 1.5 #default values for classes not specified

csv_file_loc = local_lc_folder + 'class_speed_modifiers.csv'

df_comma = pd.read_csv(csv_file_loc, nrows=1,sep=",")
df_semi = pd.read_csv(csv_file_loc, nrows=1, sep=";")

if df_comma.shape[1]>df_semi.shape[1]:
    modifiers =  pd.read_csv(csv_file_loc, sep=',')
else:
    modifiers =  pd.read_csv(csv_file_loc, sep=';')

for class_num in range(0,len(modifiers)):
    lookup[int(modifiers.loc[class_num,'raster_value'])] = modifiers.loc[class_num,'modifier']
    if force_rivers == 1:
        if modifiers.loc[class_num,'water'] == 'yes':
            water_divider = modifiers.loc[class_num,'modifier'] # for when rivers are forced

In [ ]:
## Rivers and bridges as obstacles

if force_rivers == 1:

    rivers_file = sorted([os.path.join(local_water_folder,file) \
            for file \
            in os.listdir(local_water_folder) \
            if file.endswith(".shp")])

    rivers_file = rivers_file[0]

    # local file import
    rivs = gpd.read_file(rivers_file,driver="ESRI Shapefile")

    # minor cleanup
    rivs = rivs.reset_index()
    rivs_slim = rivs[['geometry']]
    rivs_slim['exist'] = 1
    rivs_slim = rivs_slim.to_crs(dest_crs)

    # create a generator containing geometry, value pairs for rivers

    riv_shapes = ((geom,exist) for geom, exist in zip(rivs_slim.geometry,rivs_slim['exist']))

    # This will give the raster the size and dimensions of the landcover raster -- areas not covered by rivers will be 0.
        
    riv_rast = features.rasterize(riv_shapes,\
                      out_shape = (lc_profile['height'],\
                                   lc_profile['width']),\
                      transform=lc_profile['transform'],
                      all_touched=True,
                      fill=0,
                      dtype = np.float32)

In [ ]:
#read a lc file to get the right export profile
if seasonal_lc ==0:
    profile_load_location = lc_file_generic
else:
    profile_load_location = lc_folder_final+seasons[0]+'.tif'
    
with rasterio.open(profile_load_location) as lc_src:
        # Read as numpy array
        lc_profile = lc_src.profile

## Roads to walking surface mask raster
#We assume that people walking on roads and paths are not affected by landcover. To model this we turn roads into a raster with value = 1 (for 1 * speed). Then we merge it with the landcover raster for a final walking speed modifier raster

roads_file = local_roads_folder + 'final_roads.gpkg'

rds = gpd.read_file(os.path.join(roads_file),driver="gpkg")

# assign 1 value to represent existence of road
rds['exist'] = 1

# generator of vector shapes and values (boolean)
rds_shapes = ((geom,exist_val) for geom, exist_val in zip(rds.geometry,rds['exist']))

# This will give the raster the size and dimensions of the landcover raster -- areas not covered by roads will be 0.

rd_mask_rast = features.rasterize(rds_shapes,\
                  out_shape = (lc_profile['height'],\
                               lc_profile['width']),\
                  transform=lc_profile['transform'],
                  all_touched=True,
                  fill=0,
                  dtype = np.uint8)

In [ ]:
## Base walking speeds from DEM

#### DEM to slope

#First import the DEM and transform it to the same CRS, cell resolution, and dimensions as the landcover layer. This enables raster math between the layers and any other arrays derived from them.

dem_file = sorted([os.path.join(local_dem_folder,file) \
            for file \
            in os.listdir(local_dem_folder) \
            if file.endswith(".tif")])

dem_file = dem_file[0]

with rasterio.open(dem_file) as dem_src:
    # Read as numpy array
    dem_array = dem_src.read(1)
    dem_transform = dem_src.transform
    dem_profile = dem_src.profile

#dem_array_reproj.profile
dem_array = np.round(dem_array,0).astype(np.float32)

# use get_slope function from common_rasterio_ops
slope = rast_ops.get_slope(dem_array,mode='fraction')

# remove artefacts that will produce slopes > 100%
slope = np.where(slope>1,1,slope)

#Calculate walking speeds over the slope using Irmischer and Clarke's 2018 walking speed formula.
# Irmischer and Clarke have a generic off-road speed formula but we don't use this given that we adjust by specific landcover type.  Rather, we modify their on-road speed.
# We include the I+C off-road formula below for reference

# walkspeed_offroad = (0.11 + (0.67 * np.exp(-np.square((slope*100) + 2) / 3600))) * 3.6 # I-C off-road
walkspeed_onroad = (0.11 + np.exp(-np.square((slope*100) + 5) / 3600)) * 3.6

walkspeed_base = walkspeed_onroad
# walkspeed_base = np.where(rd_mask_rast == 1,walkspeed_onroad,walkspeed_offroad) # included for reference purposes, in situations where you don't want to adjust by landcover

#save memory
del walkspeed_onroad

#### Vertical distances
#Calculate the additional vertical distance covered when crossing a cell (the rise, in addition to the run represented by the cell's resolution).

vert_dist_simple = 1 / np.cos(slope)

#Calculate the additional distance associated with zig-zagging paths - the zig goes sideways halfway up the cell, the zag sideways up the other half. We do not consider circumstances with more than 2 zig zags per cell -- possibly problematic if using large cells (1km+)
# The switchback cutoff value is somewhat arbitrary and perhaps even varies by culture / context. 
# We use one of the higher values found in the literature as residents of the Himalayas might be expected to have a high tolerance for walking up steep hills

switchback_cutoff = 0.20 #0.3 for himalayas
vert_dist_switchback = np.tan(slope) / np.sin(switchback_cutoff)

#Combine the two arrays into one walking cost array, forcing walkers to use zig-zagging switchbacks while crossing terrain above a cutoff slope of `30%` (0.30). 

vert_dist_switchback = np.where(slope <= switchback_cutoff,vert_dist_simple,vert_dist_switchback)

#save memory
del slope

# make float32 to reduce file sizes on export
# vert_dist_simple = vert_dist_simple.astype(np.float32)
del vert_dist_simple
vert_dist_switchback = vert_dist_switchback.astype(np.float32)

# Create a profile for clipping the layers

export_profile = lc_profile.copy()
export_profile.update({"dtype":'float32',\
                       "COMPRESS":'ZSTD',
                       "NUM_THREADS":'ALL_CPUS',
                       "nodata":-99999})

vert_dist_switchback_mask, vert_dist_switchback_mask_tform = rast_ops.clip_in_memory(vert_dist_switchback,export_profile, aoi.geometry)

export_profile_switch = export_profile.copy()
export_profile_switch['transform'] = vert_dist_switchback_mask_tform
export_profile_switch['width'] = vert_dist_switchback_mask.shape[1]
export_profile_switch['height'] = vert_dist_switchback_mask.shape[0]

with rasterio.open(
        os.path.join(local_dem_folder,'vert_dist_switchback.tif'), 'w',**export_profile_switch) as dst:
    dst.write(vert_dist_switchback_mask, indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest)

del vert_dist_switchback_mask

In [ ]:
if seasonal_lc == 0:

    with rasterio.open(lc_file_generic) as lc_src:
            # Read as numpy array
            lc_array = lc_src.read()
            lc_profile = lc_src.profile
            lc_transform = lc_src.transform

    lc_array = lc_array.astype(np.int8)
    
    # Reclassify in a single operation using broadcasting
    lc_array = lookup[lc_array].astype(np.float16)

for season_num in range(0, len(seasons)):
    
    current_season = seasons[season_num]

    if seasonal_lc == 1:
        with rasterio.open(os.path.join(lc_folder_final,current_season+'.tif')) as lc_src:
            # Read as numpy array
            lc_array = lc_src.read()
            lc_profile = lc_src.profile
            lc_transform = lc_src.transform

        lc_array = lc_array.astype(np.int8)
    
        # Reclassify in a single operation using broadcasting
        lc_array = lookup[lc_array].astype(np.float16)

    #First combine the rivers with the landcover raster, inserting a `600000` divider where rivers exist, so crossing rivers without a bridge has a huge cost. Then combine with the road mask, inserting a `1` multiplier where roads are. The order is important, so roads overwrite rivers (implicitly via bridges, which are not reliably recorded in many roads datasets)
    #</br></br>Note that if landcover *multipliers* instead of *dividers* are used, you need to invert this and use a very small decimal value for the rivers.

    if force_rivers == 1:
        # force dividers on water from shapefiles, in addition to satellite imagery
        walkspeed_mod_rast = np.where(riv_rast == 1, water_divider, lc_array)
        # del riv_rast
    else:
        walkspeed_mod_rast = lc_array 

    walkspeed_mod_rast = walkspeed_mod_rast[0, :, : ]
    
    # del lc_array
    walkspeed_mod_rast = walkspeed_mod_rast.astype(np.float16)
    walkspeed_mod_rast = np.round(walkspeed_mod_rast,1).astype(np.float16)

    # treat roads as bridging rivers by default
    walkspeed_mod_rast = np.where(rd_mask_rast == 1, 1, walkspeed_mod_rast).astype(np.float16)
        
    ## Merge rasters into final walking friction surface

    # Combine the various arrays into a final walking friction surface in 6 stages:
    # 1. Multiply the base walking speed computed from the DEM Slope by the speed modifier
    # 2. Create a monsoon walking speed as 0.75 of the base walking speed and the winter walking speed similarly, using a multiplier determined by elevation
    # 3. Adjust the speeds for altitude
    # 4. Transform these speeds into friction values
    # 5. Multiply the friction values by the vert/horizontal multiplication factor (e.g. 1.5)
    # 6. Convert extraneous values to -99999 nodata values
    
    walkspeed_step1 = np.divide(walkspeed_base,walkspeed_mod_rast)
    txt_file = local_lc_folder + current_season + '.txt'

    with open(txt_file, 'r') as file:
                # Read the first line
                seasonal_walk_multiplier = float(file.readline().strip())
    
    walkspeed_step1 = np.multiply(walkspeed_step1,seasonal_walk_multiplier)

    walkspeed_step1 = walkspeed_step1.astype(np.float32)

    # Adjust walkspeeds by altitude
    # We adjust altitude in two steps based on a literature review into how lower oxygen content at altitude affects walking speeds. Note this is not the best documented subject, at least in terms we can computer into a friction surface.
    # This formula could probably be streamlined so that this step is condensed into one move
    # The Global Friction Surface has just one formula but I found its high altitude (>5000) modifiers to be a little low compared to the available literature on athletic performance at altitude. Not a big deal except if you're working in the Himalayas

    alt_adjust_under3k = np.where(dem_array <= 2350, walkspeed_step1, ((walkspeed_step1) / (1 + ((dem_array - 2350)/5000))) )
    walkspeed_step2 = np.where(dem_array <= 3000, alt_adjust_under3k, ((walkspeed_step1) / (0.323 * np.exp((.00042*dem_array)))) )

    # save memory 
    del alt_adjust_under3k, walkspeed_step1

    # Refactor walking speeds to friction values in units of cell size / hour (e.g. 30m / hour)
    # To prepare by minute instead just multiply by 60
    friction_walk_step1 = (1 / walkspeed_step2) / (1000 / lc_transform.a)

    # save memory
    del walkspeed_step2

    # Now multiply the friction surface by the merged vertical/horizontal distance to calculate the final friction surface
    friction_walk_final = np.multiply(friction_walk_step1,vert_dist_switchback)

    # Weed out Inf values and super high river values
    # We use 1 as an arbitrary cutoff on the assumption that it will never actually take 1 hour to cross a grid cell, so values above that are bogus and filterable

    friction_walk_final = np.where(friction_walk_final > 1, 1, friction_walk_final)
    friction_walk_final = np.round(friction_walk_final,8).astype(np.float32)       
    friction_walk_final_mask, friction_walk_mask_tform = rast_ops.clip_in_memory(friction_walk_final,export_profile, aoi.geometry)
    
    export_profile2 = export_profile.copy()
    export_profile2['transform'] = friction_walk_mask_tform
    export_profile2['width'] = friction_walk_final_mask.shape[1]
    export_profile2['height'] = friction_walk_final_mask.shape[0]
    
    with rasterio.open(
            os.path.join(fric_dir,current_season+'_walk.tif'), 'w',**export_profile2) as dst:
        dst.write(friction_walk_final_mask, indexes=1)
        dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest)